In [1]:
import numpy as np
# import tensorflow as tf
from tensorflow import keras
# import tensorflow
from tensorflow.keras.layers import BatchNormalization, Dense, Input, BatchNormalization
from tensorflow.keras import layers, Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import GaussianNoise


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization
from tensorflow.keras.layers import GaussianNoise

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, BatchNormalization, GaussianNoise, InputLayer
from tensorflow.keras.callbacks import ModelCheckpoint

import matplotlib.pyplot as plt

2023-10-16 09:36:06.458988: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [2]:
import os
# os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'

os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [3]:
def autoenco(k,n,SNRs):
    BER = []
    R = float(k/n)
    Nt = 10000
    M = 2**k


    label = np.random.randint(M,size=Nt)
    data = [] # one-hot encodedno


    for i in label:
        temp = np.zeros(M)
        temp[i] = 1
        data.append(temp) 


    data = np.array(data)



    input_signal = Input(shape=(M,))

    encoded = Dense(M, activation='relu')(input_signal)
    encoded1 = Dense(n, activation='linear')(encoded)
    encoded2 = BatchNormalization()(encoded1)

    EbNo_train = 10 ** (0.7)
    encoded3 = GaussianNoise(np.sqrt(1/(2*R*EbNo_train)))(encoded2)

    decoded = Dense(M, activation='relu')(encoded3)
    decoded1 = Dense(M, activation='softmax')(decoded)

    autoencoder = Model(input_signal, decoded1)
    #sgd = SGD(lr=0.001)
    autoencoder.compile(optimizer='adam', loss='categorical_crossentropy')
    print(autoencoder.summary())

    N_val = 1500
    val_label = np.random.randint(M,size=N_val)
    val_data = []
    
    
    for i in val_label:
        temp = np.zeros(M)
        temp[i] = 1
        val_data.append(temp)
    
    
    val_data = np.array(val_data)

    autoencoder.fit(data, data,epochs=30,
                  batch_size=300,
                  validation_data=(val_data, val_data),
                  callbacks=[ModelCheckpoint(save_best_only=True, 
                                             filepath="ok")])
 
    autoencoder.load_weights("ok")

    encoder = Model(input_signal, encoded2)
    encoded_input = Input(shape=(n,))

    deco = autoencoder.layers[-2](encoded_input)
    deco = autoencoder.layers[-1](deco)


    # create the decoder model
    decoder = Model(encoded_input, deco)

    N = 50000
    test_label = np.random.randint(M,size=N)
    test_data = []

    
    for i in test_label:
        temp = np.zeros(M)
        temp[i] = 1
        test_data.append(temp)

    test_data = np.array(test_data)

    for snr in SNRs:
        EbNo=10.0**(snr/10.0)
        noise_std = np.sqrt(1/(2*R*EbNo))
        noise_mean = 0
        no_errors = 0
        noise = np.random.normal(0,noise_std,(N,n))
        encoded_signal = encoder.predict(test_data)
        final_signal = encoded_signal + noise
        pred_final_signal =  decoder.predict(final_signal)
        pred_output = np.argmax(pred_final_signal,axis=1)
        no_errors = (pred_output != test_label)
        no_errors =  no_errors.astype(int).sum()
        BER.append(no_errors / N)
        
    return BER

In [4]:
# use k =3 and n = 7
SNRs = np.arange(1.0,18.0,0.5)

autoenco(3, 7, SNRs)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 8)]               0         
_________________________________________________________________
dense (Dense)                (None, 8)                 72        
_________________________________________________________________
dense_1 (Dense)              (None, 7)                 63        
_________________________________________________________________
batch_normalization (BatchNo (None, 7)                 28        
_________________________________________________________________
gaussian_noise (GaussianNois (None, 7)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 64        
_________________________________________________________________
dense_3 (Dense)              (None, 8)                 72    

2023-10-16 09:36:08.855203: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-10-16 09:36:09.143283: E tensorflow/stream_executor/cuda/cuda_driver.cc:282] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-10-16 09:36:09.143364: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: 5310f4639566
2023-10-16 09:36:09.143373: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: 5310f4639566
2023-10-16 09:36:09.143486: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 470.82.1
2023-10-16 09:36:09.143517: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 470.82.1
2023-10-16 09:36:09.143524: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 470.82.1
2023-10-16 09:36:09.161281: I tensorflow/core/platform/pro

10000/10000 [==============================] - 1s 80us/sample - loss: 2.2922 - val_loss: 2.0369
Epoch 2/30
10000/10000 [==============================] - 0s 7us/sample - loss: 1.8084 - val_loss: 1.9567
Epoch 3/30
10000/10000 [==============================] - 0s 7us/sample - loss: 1.4609 - val_loss: 1.8150
Epoch 4/30
10000/10000 [==============================] - 0s 7us/sample - loss: 1.1757 - val_loss: 1.6024
Epoch 5/30
10000/10000 [==============================] - 0s 7us/sample - loss: 0.9281 - val_loss: 1.3663
Epoch 6/30
10000/10000 [==============================] - 0s 6us/sample - loss: 0.7281 - val_loss: 1.1184
Epoch 7/30
10000/10000 [==============================] - 0s 6us/sample - loss: 0.5701 - val_loss: 0.8814
Epoch 8/30
10000/10000 [==============================] - 0s 7us/sample - loss: 0.4536 - val_loss: 0.6683
Epoch 9/30
10000/10000 [==============================] - 0s 7us/sample - loss: 0.3615 - val_loss: 0.4906
Epoch 10/30
10000/10000 [==============================]

2023-10-16 09:36:11.638958: W tensorflow/core/util/tensor_slice_reader.cc:95] Could not open ./ok: Data loss: not an sstable (bad magic number): perhaps your file is in a different file format and you need to use a different restore operator?


[0.03012,
 0.02126,
 0.01506,
 0.00976,
 0.0059,
 0.00384,
 0.00222,
 0.00118,
 0.00078,
 0.00034,
 0.00012,
 4e-05,
 6e-05,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]